##CRYPTOINSIGHTHUB

In [ ]:
import requests
import pandas as pd
import numpy as np
import time
!pip install arch
!pip install tabulate
!pip install matplotlib seaborn


In [3]:
API_KEY = "CG-e5nkKqYuns3bSAoYNrDqFY8P"   

def fetch_coin_data(coin_id, days=90, max_retries=5):
    url = f"https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart"
    params = {'vs_currency':'usd','days':days,'interval':'daily'}
    headers = {"x-cg-api-key": API_KEY}

    retries = 0
    while retries < max_retries:
        try:
            response = requests.get(url, params=params, headers=headers, timeout=10)
            response.raise_for_status()
            data = response.json()
            break  # success
        except requests.exceptions.HTTPError as e:
            if response.status_code == 429:
                retries += 1
                wait_time = 2 ** retries  # exponential backoff
                print(f"Rate limit hit, retrying in {wait_time}s...")
                time.sleep(wait_time)
            else:
                raise RuntimeError(f"Failed to fetch data for {coin_id}: {e}")
        except Exception as e:
            raise RuntimeError(f"Failed to fetch data for {coin_id}: {e}")

  # --- Prices (approx OHLC) ---
    prices = pd.DataFrame(data['prices'], columns=['Date', 'Close'])
    prices['Date'] = pd.to_datetime(prices['Date'], unit='ms')
    prices['Open'] = prices['Close']
    prices['High'] = prices['Close']
    prices['Low'] = prices['Close']

    # --- Market Cap ---
    market_cap = pd.DataFrame(data['market_caps'], columns=['Date', 'Market Cap'])
    market_cap['Date'] = pd.to_datetime(market_cap['Date'], unit='ms')

    # --- Volume ---
    volume = pd.DataFrame(data['total_volumes'], columns=['Date', 'Volume'])
    volume['Date'] = pd.to_datetime(volume['Date'], unit='ms')

    # --- Merge ---
    df = prices.merge(market_cap, on='Date').merge(volume, on='Date')
    df['Coin'] = coin_id.capitalize()

    df = df[['Coin', 'Date', 'Open', 'High', 'Low', 'Close', 'Market Cap', 'Volume']]
    df = df.sort_values('Date').reset_index(drop=True)

    return df


###Fetching Coin Data from the API

In [ ]:
btc_df = fetch_coin_data('bitcoin')
time.sleep(2)
eth_df = fetch_coin_data('ethereum')
time.sleep(2)
car_df = fetch_coin_data('cardano')
time.sleep(2)
dog_df = fetch_coin_data('dogecoin')

all_df = pd.concat([btc_df, eth_df, car_df, dog_df], ignore_index=True)

pd.set_option('display.float_format', '{:,.2f}'.format)

print(all_df.head(10).to_markdown(tablefmt="grid"))

#added delay between request inorder to avoide 429 error


In [5]:
#add_realized_volatility calculates volatility based on the historical data of the asset's price
def add_realized_volatility(df, window=30): #Rolling Standard Deviation → Realized Volatility
    df = df.sort_values("Date")
    df['LogReturn'] = np.log(df['Close'] / df['Close'].shift(1))
    df['RealizedVol'] = df['LogReturn'].rolling(window).std() * np.sqrt(365)  # represents how the market has been over a period of time
    return df


def add_atr(df, window=14): #Average True Range (ATR) → Calc Intraday Volatility
    df = df.sort_values("Date")
    df['H-L'] = df['High'] - df['Low']
    df['H-PC'] = (df['High'] - df['Close'].shift(1)).abs()
    df['L-PC'] = (df['Low'] - df['Close'].shift(1)).abs()
    df['TR'] = df[['H-L', 'H-PC', 'L-PC']].max(axis=1)
    # True Range = max of (High-Low, High-prevClose, Low-prevClose)
    df['ATR'] = df['TR'].rolling(window).mean()# Rolling ATR helps to measure risk and stoploss
    return df


In [21]:
from arch import arch_model

#def estimate_garch(df):
  #  df = df.sort_values("Date")
  #  df['LogReturn'] = np.log(df['Close'] / df['Close'].shift(1))

    # Scale returns so optimizer behaves better
   # scaled = df['LogReturn'].dropna() * 100

    #model = arch_model(scaled, vol='Garch', p=1, q=1)
    #res = model.fit(disp="off")

    # Convert conditional volatility back to original scale
    #df['GARCH_Vol'] = res.conditional_volatility / 100 
    #the estimated volatility at each time point according to the GARCH model..
    #Dividing by 100 converts it back to the original scale
    #return df, res

def estimate_garch(df):
    from arch import arch_model
    returns = df['LogReturn'].dropna() * 100  # percentage returns
    am = arch_model(returns, vol='Garch', p=1, q=1)
    res = am.fit(disp='off')
    # create full length series with NaN for first missing returns
    garch_vol = pd.Series(index=df.index, data=np.nan)
    garch_vol.iloc[1:] = res.conditional_volatility / 100  # back to fraction
    return df, garch_vol

In [22]:
btc_df = add_realized_volatility(btc_df)
btc_df = add_atr(btc_df)
btc_df, btc_garch = estimate_garch(btc_df)

eth_df = add_realized_volatility(eth_df)
eth_df = add_atr(eth_df)
eth_df, eth_garch = estimate_garch(eth_df)

car_df = add_realized_volatility(car_df)
car_df = add_atr(car_df)
car_df, car_garch = estimate_garch(car_df)

dog_df = add_realized_volatility(dog_df)
dog_df = add_atr(dog_df)
dog_df, dog_garch = estimate_garch(dog_df)



In [8]:
all_df = pd.concat([btc_df, eth_df, car_df, dog_df], ignore_index=True)
all_df = all_df.sort_values(['Coin', 'Date']).reset_index(drop=True)


In [ ]:
print(car_df.head().to_markdown(tablefmt="grid"))

In [26]:
import plotly.graph_objects as go
from datetime import timedelta

# List of coins and their dataframes
coins = [
    
    ("CAR", car_df, car_garch if 'car_garch' in globals() else None)
]

fig = go.Figure()

# Iterate through coins
for name, df, garch in coins:
    # Filter last 90 days
    last_90_days = df['Date'].max() - timedelta(days=90)
    df_recent = df[df['Date'] >= last_90_days]

    # Price trace
    fig.add_trace(go.Scatter(
        x=df_recent['Date'],
        y=df_recent['Close'],
        mode='lines',
        name=f'{name} Price',
        yaxis='y1',
        line=dict(width=2)
    ))

    # Realized Volatility
    fig.add_trace(go.Scatter(
        x=df_recent['Date'],
        y=df_recent['RealizedVol'],
        mode='lines',
        name=f'{name} RealizedVol',
        yaxis='y2',
        line=dict(width=2, dash='solid')
    ))

    # ATR
    fig.add_trace(go.Scatter(
        x=df_recent['Date'],
        y=df_recent['ATR'],
        mode='lines',
        name=f'{name} ATR',
        yaxis='y2',
        line=dict(width=2, dash='dot')
    ))

    # Optional: GARCH
    if garch is not None:
        fig.add_trace(go.Scatter(
            x=df_recent['Date'],
            y=garch[-len(df_recent):],  # ensure alignment
            mode='lines',
            name=f'{name} GARCH',
            yaxis='y2',
            line=dict(width=2, dash='dash')
        ))

# Layout with dual y-axes
fig.update_layout(
    title='Cardano Price and Volatility (Last 90 Days)',
    xaxis=dict(title='Date'),
    yaxis=dict(title='Price', side='left'),
    yaxis2=dict(title='Volatility', overlaying='y', side='right'),
    template='plotly_white',
    legend=dict(x=0.01, y=0.99),
    hovermode='x unified',
    width=1200,
    height=600
)

fig.show()


In [29]:
import plotly.graph_objects as go
from datetime import timedelta

# List of coins and their dataframes
coins = [
    
    ("CARDANO", car_df, car_garch if 'car_garch' in globals() else None)
]

fig = go.Figure()

# Iterate through coins
for name, df, garch in coins:
    # Filter last 90 days
    last_90_days = df['Date'].max() - timedelta(days=90)
    df_recent = df[df['Date'] >= last_90_days]

    

    # ATR
    fig.add_trace(go.Scatter(
        x=df_recent['Date'],
        y=df_recent['ATR'],
        mode='lines',
        name=f'{name} ATR',
        yaxis='y2',
        line=dict(width=2, dash='dot')
    ))

    # Optional: GARCH
    if garch is not None:
        fig.add_trace(go.Scatter(
            x=df_recent['Date'],
            y=garch[-len(df_recent):],  # ensure alignment
            mode='lines',
            name=f'{name} GARCH',
            yaxis='y2',
            line=dict(width=2, dash='dash')
        ))

# Layout with dual y-axes
fig.update_layout(
    title='Cardano Price and Volatility (Last 90 Days)',
    xaxis=dict(title='Date'),
    yaxis=dict(title='Price', side='left'),
    yaxis2=dict(title='Volatility', overlaying='y', side='right'),
    template='plotly_white',
    legend=dict(x=0.01, y=0.99),
    hovermode='x unified',
    width=1200,
    height=600
)

fig.show()
